# SCAPY
18/09/2024

## Cours
Ethernet -> MAC  
IP -> adresses IP  
Transport -> port  

## Code

#### Import

In [8]:
from scapy.all import *

#### Connaitre les interfaces disponibles

In [ ]:
# Affiche toutes les interfaces réseau disponibles
print(conf.ifaces)

In [ ]:
from scapy.all import *
sniff(count=10)

#### Voir les trames

TTL : 64 passages de routeur, (traceroute travaille dessus)

In [ ]:

def maFonction(trame):
    """
    Voir les trames
    """
    print(trame)
    print(trame.show()) # Détaille des trames

sniff(iface="Software Loopback Interface 1", prn=maFonction, count=15)


Filtrer :

In [ ]:

def maFonction(trame):
    print(trame.show()) # Détaille des trames

sniff(iface="Software Loopback Interface 1", prn=maFonction, filter="tcp", count=15)

In [ ]:

def maFonction(trame):
    print(trame.show()) # Détaille des trames

sniff(iface="Software Loopback Interface 1", prn=maFonction, filter="ip and tcp 10.10.1.2", count=15)

Filtrage BPF : Berkeley Packet Filter  
* Filtrage rapide mais peu flexible

In [ ]:

def maFonction(trame):
    print(trame.show()) # Détaille des trames

sniff(iface="Software Loopback Interface 1", prn=maFonction, filter="ip and tcp and src net 192.168.1.0/24", count=15)

Filtrage par fonction

In [ ]:

def maFonction(trame):
    print(trame.show()) # Détaille des trames

def monFiltre(pkt):
    if(pkt.haslayer(TCP)): # Si trame à une couche TCP
        return True
    else:
        return False

sniff(iface="Software Loopback Interface 1", prn=maFonction, lfilter=monFiltre, count=15)

Adresse ip source  
Adresse Mac source

In [27]:

def maFonction(trame):
    print("L'adresse ip source est :", trame[IP].src) # Détaille des trames
    #print("Trame TCP avec port 443 : ", trame[TCP].dport==443) # Détaille des trames
    #print("L'adresse Mac source est :", trame[Ether].src) # Détaille des trames
    #print("L'applicatif : ", trame[RAW].load)

def monFiltre(pkt):
    if(pkt.haslayer(TCP)): # Si trame à une couche TCP
        return True
    else:
        return False

sniff(iface="Software Loopback Interface 1", prn=maFonction, filter="tcp" ,lfilter=monFiltre, count=15)

L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1
L'adresse ip source est : 127.0.0.1


<Sniffed: TCP:15 UDP:0 ICMP:0 Other:0>